## Домашнее задание № 2

В рамках этого домашнего задания вам предлагается пройти весь цикл машинного обучения, начиная от заполнения пропусков и кодирования категориальных переменных, заканчивая обучением модели и оценкой ее качества. В помощь вам представлен baseline в виде дерева решений на нескольких признаках.

In [33]:
import pandas as pd 
import matplotlib.pyplot as plt # рисовалка графиков
import seaborn as sns # еще одна рисовалка графиков
from sklearn.tree import DecisionTreeClassifier # дерево решений для задачи классификации
from sklearn.ensemble import RandomForestClassifier # случайный лес - набор деревьев, обученных на случайных подвыборках
from sklearn.metrics import ( # метрики качества наших моделей
accuracy_score, # процент правильных ответов
precision_score, # точность (сколько людей из тех, что мы отнесли к погибшим, действительно погибли)
recall_score, # полнота (сколько из всех погибших людей наш алгоритм нашел)
f1_score, # f-мера - среднее гармоническое между точностью и полнотой
) 
from sklearn.model_selection import train_test_split # разделение на обучающую и тестовую выборки

import warnings 
warnings.simplefilter('ignore') # игнорировать предупреждения

Загрузим данные:

In [34]:
df = pd.read_csv('../../data/titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Есть ли пропуски в данных?

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Видим, что пропущены значения в колонке `Age`, `Cabin` и `Embarked`. Заполним пропуски в колонке с возрастом средним значением, а в колонках `Cabin` и `Embarked` - самым частым.

In [36]:
mean_age = df['Age'].mean()
df['Age'] = df['Age'].fillna(mean_age)

cabin_mode = df['Cabin'].mode()[0]
df['Cabin'] = df['Cabin'].fillna(cabin_mode)

embarked_mode = df['Embarked'].mode()[0]
df['Embarked'] = df['Embarked'].fillna(embarked_mode)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          891 non-null object
Embarked       891 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Все, пропуски заполнены (не самым умным образом, для признака `Cabin`, у которого было очень много пропусков, можно попробовать закодировать пропуски новой категорией `unknown`, или просто выкинуть его из рассмотрения), можно переходить к кодированию категориальных признаков. Сделаем очень просто, закодируем все категории числами.

Выкинем ненужные колонки:

In [40]:
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

In [32]:
from sklearn.preprocessing import OrdinalEncoder

In [41]:
enc = OrdinalEncoder()
transformed = enc.fit_transform(df[['Sex', 'Ticket', 'Cabin', 'Embarked']])
transformed

array([[  1., 523.,  47.,   2.],
       [  0., 596.,  81.,   0.],
       [  0., 669.,  47.,   2.],
       ...,
       [  0., 675.,  47.,   2.],
       [  1.,   8.,  60.,   0.],
       [  1., 466.,  47.,   1.]])

Переведем получившийся массив в таблицу:

In [42]:
transformed_df = pd.DataFrame(transformed, 
                              columns=['Sex_coded',
                                       'Ticket_coded',
                                       'Cabin_coded',
                                       'Embarked_coded']
                             )

In [43]:
df = pd.concat([df, transformed_df], axis=1) # соединим исходную таблицу и закодированные признаки
df.drop(['Sex', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True) # удалим исходные незакодированные признаки

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived          891 non-null int64
Pclass            891 non-null int64
Age               891 non-null float64
SibSp             891 non-null int64
Parch             891 non-null int64
Fare              891 non-null float64
Sex_coded         891 non-null float64
Ticket_coded      891 non-null float64
Cabin_coded       891 non-null float64
Embarked_coded    891 non-null float64
dtypes: float64(6), int64(4)
memory usage: 69.7 KB


Теперь, когда у нас все признаки числовые, разделим выборку на train и test.

In [45]:
X = df.drop('Survived', axis=1)
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3, # доля данных на тест
                                                    stratify=y # сохранить баланс классов в обучающей и тестовой выборках
                                                   )

Создадим дерево.

In [46]:
tree = DecisionTreeClassifier(max_depth=100, max_leaf_nodes=5)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
                       max_features=None, max_leaf_nodes=5,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [47]:
y_train_pred = tree.predict(X_train)
print('Accuracy score: ', accuracy_score(y_train, y_train_pred))
print('Precision score: ', precision_score(y_train, y_train_pred))
print('Recall score: ', recall_score(y_train, y_train_pred))

Accuracy score:  0.8250401284109149
Precision score:  0.768595041322314
Recall score:  0.7782426778242678


Показатели неплохие, точность в 82 процента это неплохо.

In [48]:
y_test_pred = tree.predict(X_test)
print('Accuracy score: ', accuracy_score(y_test, y_test_pred))
print('Precision score: ', precision_score(y_test, y_test_pred))
print('Recall score: ', recall_score(y_test, y_test_pred))

Accuracy score:  0.7574626865671642
Precision score:  0.6862745098039216
Recall score:  0.6796116504854369


Точность на тестовой выборке чуть пониже, как и остальные метрики, это значит, что дерево переобучилось, можно понастраивать его параметры вручную, чтобы добиться более хорошего качества на тестовой выборке.

Процесс настройки параметров модели для снижения переобучения называется __регуляризацией__.